In [1]:
%pip install PyniteFEA[all]
%pip install Pynite
%pip install trimesh


   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ----------- ---------------------------- 3.4/12.3 MB 18.3 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 18.9 MB/s eta 0:00:01
   ---------------------------------- ----- 10.7/12.3 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 17.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 12.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/821.6 kB ? eta -:--:--
   --------------------------------------- 821.6/821.6 kB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.9 MB ? eta -:--:--
   ------------------------- -------------- 3.1/4.9 MB 15.4 MB/s eta 0:00:01
   ---------------------------------------- 4.9/4.9 MB 14.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   -------------- --------

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 7.8 MB/s eta 0:00:00
  Created wheel for Pynite: filename=pynite-1.4.2-py3-none-any.whl size=3754 sha256=7dd8f24ff1a2135832240d9d35b52b28800be1ef0704ac7e36d09076b898bc5b
  Stored in directory: c:\users\legion\appdata\local\pip\cache\wheels\cf\b0\a5\2d5af2541abe8814dd4964515ef23e17687e8250c196c22096
Successfully built Pynite
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


PyNite Script for Multi-Span Steel Frame with Uniform Slab Loads
To model a simple steel frame (multi-span beam) supporting a concrete slab, we can use the PyNite FEA library in Python. The following script lets you define multiple spans and members, apply uniform Dead Load (DL) and Live Load (LL) on all beams (simulating slab weight and live load), and then analyze the structure under the standard 1.2 DL + 1.6 LL load combination.

In [ ]:
# Import the FEModel3D class from PyNite
from Pynite import FEModel3D

# Initialize a new 3D finite element model
model = FEModel3D()


Define Material and Section Properties
Next, define the material (steel) and cross-section properties for the beams. Here we use typical properties for structural steel.

In [ ]:
# --- Material properties (Steel) ---
# Users can edit these values if using a different material or units.
E = 210e6   # Young's modulus in kN/m^2 (≈210 GPa, typical for steel)
nu = 0.3    # Poisson's ratio (steel ~0.3)
G = E / (2 * (1 + nu))  # Shear modulus in kN/m^2
rho = 77.0e-6  # Density in kN/m^3 (approx 77 kN/m^3, not critical unless self-weight is used)

# Add steel material to the model (name, E, G, nu, rho)
model.add_material('Steel', E, G, nu, rho)

# --- Cross-section properties for the steel beams ---
# Users can edit these to match a specific beam section (A: area, Iy/Iz: moments of inertia, J: torsional constant).
A  = 0.02      # Cross-sectional area in m^2 (example value)
Iy = 8.3e-6    # Moment of inertia about local y-axis (m^4) - e.g., weak axis
Iz = 4.2e-5    # Moment of inertia about local z-axis (m^4) - e.g., strong axis (vertical bending axis)
J  = 1.0e-5    # Torsional constant (m^4) - resistance to twisting

# Add the section properties to the model
model.add_section('BeamSection', A, Iy, Iz, J)


Define Geometry: Nodes and Members (Spans)

In [ ]:
# --- Define span lengths (in meters) ---
# EDIT HERE: Modify this list to change the number of spans and their lengths.
span_lengths = [6.0, 6.0, 6.0]  # Example: three spans, each 6.0 m long

# Automatically generate node coordinates for the spans.
# We'll assume a straight horizontal beam (along global X-axis) at Y=0, Z=0 for simplicity.
current_x = 0.0
node_names = []  # to keep track of node naming
for i, L in enumerate(span_lengths):
    # Define node at the start of the span (for first span, this is the left end)
    if i == 0:
        node_name = f"N{i}"  # e.g., "N0"
        model.add_node(node_name, X=current_x, Y=0.0, Z=0.0)
        node_names.append(node_name)
    # Define node at the end of this span
    current_x += L  # advance x by the span length
    node_name = f"N{i+1}"  # e.g., "N1", "N2", ...
    model.add_node(node_name, X=current_x, Y=0.0, Z=0.0)
    node_names.append(node_name)

# Add beam members between consecutive nodes for each span
for i in range(len(span_lengths)):
    mem_name = f"M{i+1}"  # Name members as M1, M2, ...
    start_node = f"N{i}"
    end_node   = f"N{i+1}"
    # Connect the nodes with a steel beam member
    model.add_member(mem_name, i_node=start_node, j_node=end_node,
                     material_name='Steel', section_name='BeamSection', 
                     rotation=0.0)
    # (rotation=0 aligns the member's local axes with global axes by default)


Define supports at each node

In [ ]:
# --- Define supports at each node ---
num_nodes = len(node_names)
for idx, node_name in enumerate(node_names):
    if idx == 0:
        # Left end support (Pinned: all translations fixed, rotations free)
        model.def_support(node_name, support_DX=True, support_DY=True, support_DZ=True,
                                      support_RX=False, support_RY=False, support_RZ=False)
    elif idx == num_nodes - 1:
        # Right end support (Roller: free in global X, fixed in vertical Y and Z; 
        # also fix torsional rotation RX for stability)
        model.def_support(node_name, support_DX=False, support_DY=True, support_DZ=True,
                                      support_RX=True, support_RY=False, support_RZ=False)
    else:
        # Intermediate supports (Pin: treat like pinned supports at column tops)
        model.def_support(node_name, support_DX=True, support_DY=True, support_DZ=True,
                                      support_RX=False, support_RY=False, support_RZ=False)


In [ ]:
# --- Load cases and uniform load values ---
# EDIT HERE: Adjust the load intensities as needed (in kN/m in this example).
DL_intensity = 5.0   # Dead load (e.g. slab self-weight + finishes) in kN/m
LL_intensity = 2.5   # Live load (e.g. occupancy load) in kN/m

# Define load case names for clarity
DL_case = 'DL'
LL_case = 'LL'

# Add distributed loads to each beam member for each load case
for i, L in enumerate(span_lengths):
    mem_name = f"M{i+1}"
    # Dead Load: uniform over full span (x1=0, x2=L)
    model.add_member_dist_load(member_name=mem_name, direction="FY",
                               w1=-DL_intensity, w2=-DL_intensity, 
                               x1=0.0, x2=L, case=DL_case)
    # Live Load: uniform over full span
    model.add_member_dist_load(member_name=mem_name, direction="FY",
                               w1=-LL_intensity, w2=-LL_intensity, 
                               x1=0.0, x2=L, case=LL_case)


In [ ]:
# --- Define load combination for ultimate design ---
combo_name = '1.2DL+1.6LL'
model.add_load_combo(name=combo_name, 
                     factors={DL_case: 1.2, LL_case: 1.6})


In [ ]:
# --- Run linear static analysis for the defined load combination ---
model.analyze_linear(check_stability=True, check_statics=True)


In [ ]:
# --- Results Visualization ---

# 1. Shear force diagrams (vertical shear 'Fy' along each member)
for member in model.members.values():
    member.plot_shear(direction='Fy', combo_name=combo_name, n_points=50)

# 2. Bending moment diagrams (major axis bending 'Mz' for each member)
for member in model.members.values():
    member.plot_moment(direction='Mz', combo_name=combo_name, n_points=50)

# 3. Deflected shape diagrams (vertical deflection 'dy' for each member)
for member in model.members.values():
    member.plot_deflection(direction='dy', combo_name=combo_name, n_points=50)

# 4. Support reactions: print vertical reaction at each support node for the combo
print("Support Reactions under combo", combo_name)
for i in range(len(node_names)):
    node = model.nodes[f"N{i}"]
    # RxnFY is the reaction force in global Y at node. It returns a dict of values per combo.
    reaction = node.RxnFY[combo_name]  # get the vertical reaction for our combo
    print(f"  Node {node.name}:  RY = {reaction:.2f} kN")


In [ ]:
Support Reactions under combo 1.2DL+1.6LL
  Node N0:  RY = XX.XX kN
  Node N1:  RY = YY.YY kN
  Node N2:  RY = ZZ.ZZ kN
  Node N3:  RY = WW.WW kN
